In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:
print(tf.__version__)

2.14.0


In [3]:
model = keras.models.load_model('dino_dragon_10_0.899.h5')

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('dino_dragon.tflite', 'wb') as f_out:
    f_out.write(tflite_model)

INFO:tensorflow:Assets written to: /var/folders/mz/9brv5k350y9dd73hr1vl2l5w0000gn/T/tmpn5nv746f/assets


INFO:tensorflow:Assets written to: /var/folders/mz/9brv5k350y9dd73hr1vl2l5w0000gn/T/tmpn5nv746f/assets
2023-10-17 21:13:35.312587: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2023-10-17 21:13:35.312599: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2023-10-17 21:13:35.312973: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /var/folders/mz/9brv5k350y9dd73hr1vl2l5w0000gn/T/tmpn5nv746f
2023-10-17 21:13:35.313354: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2023-10-17 21:13:35.313359: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /var/folders/mz/9brv5k350y9dd73hr1vl2l5w0000gn/T/tmpn5nv746f
2023-10-17 21:13:35.314272: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:382] MLIR V1 optimization pass is not enabled
2023-10-17 21:13:35.314656: I tensorflow/cc/saved_model/load

In [4]:
import tensorflow.lite as tflite


In [5]:
interpreter = tflite.Interpreter(model_path='dino_dragon.tflite')
interpreter.allocate_tensors()
    

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [6]:
interpreter.get_input_details()

[{'name': 'serving_default_conv2d_input:0',
  'index': 0,
  'shape': array([  1, 150, 150,   3], dtype=int32),
  'shape_signature': array([ -1, 150, 150,   3], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [7]:
interpreter.get_output_details()


[{'name': 'StatefulPartitionedCall:0',
  'index': 13,
  'shape': array([1, 1], dtype=int32),
  'shape_signature': array([-1,  1], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [8]:
input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']
input_index, output_index

(0, 13)

In [9]:
def preprocess_image(image,target_size=(150,150)):
    img = Image.open(image)
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    img_arr = np.array(img) / 255.0  # Normalize pixel values to [0, 1]
    return img_arr

In [10]:
from PIL import Image
img_path = 'Smaug_par_David_Demaret.jpg'

prep_img = preprocess_image(img_path)
prep_img

array([[[0.55294118, 0.31764706, 0.17647059],
        [0.57254902, 0.34117647, 0.17647059],
        [0.56470588, 0.3254902 , 0.14901961],
        ...,
        [0.07058824, 0.02745098, 0.01960784],
        [0.07058824, 0.03921569, 0.02745098],
        [0.10980392, 0.04705882, 0.04705882]],

       [[0.51372549, 0.28235294, 0.14901961],
        [0.55294118, 0.30980392, 0.15686275],
        [0.58039216, 0.34901961, 0.18431373],
        ...,
        [0.05882353, 0.04313725, 0.03137255],
        [0.05490196, 0.02352941, 0.01176471],
        [0.09803922, 0.03529412, 0.03921569]],

       [[0.50588235, 0.29411765, 0.15294118],
        [0.56862745, 0.34117647, 0.19607843],
        [0.55686275, 0.32156863, 0.17647059],
        ...,
        [0.0745098 , 0.02745098, 0.02745098],
        [0.09019608, 0.03529412, 0.03137255],
        [0.0745098 , 0.03529412, 0.03137255]],

       ...,

       [[0.29803922, 0.41960784, 0.36078431],
        [0.4       , 0.36078431, 0.35294118],
        [0.42745098, 0

In [11]:

X = np.float32(np.array([prep_img]))

In [12]:
interpreter.set_tensor(input_index, X)


In [13]:
interpreter.invoke()
preds = interpreter.get_tensor(output_index)

In [14]:
preds

array([[0.82448524]], dtype=float32)